## Objective is to get Category of the product when the name of product is entered

In [4]:
# Importing Packages
import numpy as np 
import pandas as pd 
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
import gensim
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from simpletransformers.language_representation import RepresentationModel
#from sklearn.naive_bayes import MultinomialNB

ModuleNotFoundError: No module named 'torch'

In [5]:
pip install torch


     -------------------------------------- 162.5/162.5 MB 1.0 MB/s eta 0:00:00


## Data Preprocessing

In [46]:
# Reading the dataset file
df = pd.read_csv('Categorical_Data2.csv')

In [47]:
#display csv file
df

,Unnamed: 0,Product name,Category
0,0,HP 14 11th Gen Intel Core i5 Processor 14-inch...,Laptops
1,1,"Dell Vostro 3400 14"" (35.56 cms) FHD Anti Glar...",Laptops
2,2,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",Laptops
3,3,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",Laptops
4,4,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",Laptops
...,...,...,...
2830,3064,888 Combo of Rose-Lavender Camphor Cone 65 Gra...,Air freshners
2831,3065,Lovin Air Freshener - 234 ml (Wild Romance),Air freshners
2832,3066,"ElectroCloud Car Air Freshener Vent Clip, Esse...",Air freshners
2833,3067,"Sugandhim Rose, Champa & Chameli Air Freshener...",Air freshners


In [48]:
#getting the required data from the dataset
df = df[['Product name', 'Category']]
df

,Product name,Category
0,HP 14 11th Gen Intel Core i5 Processor 14-inch...,Laptops
1,"Dell Vostro 3400 14"" (35.56 cms) FHD Anti Glar...",Laptops
2,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",Laptops
3,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",Laptops
4,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",Laptops
...,...,...
2830,888 Combo of Rose-Lavender Camphor Cone 65 Gra...,Air freshners
2831,Lovin Air Freshener - 234 ml (Wild Romance),Air freshners
2832,"ElectroCloud Car Air Freshener Vent Clip, Esse...",Air freshners
2833,"Sugandhim Rose, Champa & Chameli Air Freshener...",Air freshners


In [49]:
df[df['Category'] == 'All in one PC']

,Product name,Category
1149,Lenovo IdeaCentre A340 23.8” FHD IPS All-in-On...,All in one PC
1150,"Lenovo IdeaCentre AIO 330 19.5"" All-in-One Des...",All in one PC
1151,HP All in One PC 20.7-inch(52.6 cm) FHD with A...,All in one PC
1152,Lenovo IdeaCentre AIO 330 19.5-inch All-in-One...,All in one PC
1153,"ASUS Vivo AiO V222, 21.5-inch (54.61 cms) FHD,...",All in one PC
...,...,...
1430,Radiant Electro 17 inch All in One Desktop Set...,All in one PC
1431,ASUS AIO / i3-10110U / 0 / HD Cam / 4GB / 1TB ...,All in one PC
1432,HP HP 570-p053in 19.45-inch All-in-One Desktop...,All in one PC
1433,Radiant Electro 17 inch All in One Desktop Set...,All in one PC


In [50]:
df[df['Category'] == 'Desktops']

,Product name,Category
856,Lenovo IdeaCentre A340 23.8-inch FHD All-in-On...,Desktops
857,Lenovo IdeaCentre 3 Desktop (AMD Athlon Silver...,Desktops
858,(Renewed) Dell Optiplex Business Class Perform...,Desktops
859,(Renewed) Dell Optiplex 3020 17-inch (43.18 cm...,Desktops
860,(Renewed) Dell Optiplex 380 17 inch (43.18 cms...,Desktops
...,...,...
1144,Zebronics Desktop Computer (Intel Core i7-860 ...,Desktops
1145,(Renewed) Dell Optiplex Business Class Perform...,Desktops
1146,TravisLappy Desktop PC CPU Computer CORE I3 32...,Desktops
1147,Capital Gadgets Desktop Computer Intel Core (i...,Desktops


In [51]:
agData = [['All in one desktop', 'All in one PC'],
         ['All in one PC', 'All in one PC'],
         ['Laptop', 'Laptops'],
         ['All in one desktops', 'All in one PC'],
         ['Desktop', 'Desktops'],
         ['Desktops', 'Desktops'],
         ['Phone', 'Phones'],
         ['Smart phone', 'Phones'],
         ['Phones', 'Phones'],
         ['Smart Phones', 'Phones'],
         ['Smart TV', 'Smart TV'],
         ['Adhesives', 'Adhesives'],
         ['Mobile Toilet', 'Mobile toilets'],
         ['Air freshners', 'Air freshners'],
         ['printer', 'Printers']]

df_ag = pd.DataFrame(agData, columns = ['Product name', 'Category'])
df = df.append(df_ag, ignore_index = True)

In [52]:
df.Category.unique()

array(['Laptops', 'Phones', 'Printers', 'Desktops', 'All in one PC',
       'Adhesives', 'Mobile toilets', 'Art and craft', 'Smart TV',
       'Air freshners'], dtype=object)

In [53]:
# Checking for null values
df.isnull().sum()

Product name    0
Category        0
dtype: int64

In [54]:
# converting the categorical data to numerical data for further modeling
label_encoder = preprocessing.LabelEncoder()
df['Category'] = label_encoder.fit_transform(df['Category']) 
df

,Product name,Category
0,HP 14 11th Gen Intel Core i5 Processor 14-inch...,5
1,"Dell Vostro 3400 14"" (35.56 cms) FHD Anti Glar...",5
2,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",5
3,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",5
4,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",5
...,...,...
2845,Smart TV,9
2846,Adhesives,0
2847,Mobile Toilet,6
2848,Air freshners,1


In [55]:
# number of categories and their frequencies
df['Category'].value_counts()

6    302
7    302
4    295
2    289
0    287
9    286
8    281
5    279
3    268
1    261
Name: Category, dtype: int64

In [56]:
# dropping duplicate records, if any
df = df.drop_duplicates()
df = df.reset_index()

In [57]:
df.drop('index', axis =1, inplace = True)
df

,Product name,Category
0,HP 14 11th Gen Intel Core i5 Processor 14-inch...,5
1,"Dell Vostro 3400 14"" (35.56 cms) FHD Anti Glar...",5
2,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",5
3,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",5
4,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",5
...,...,...
2845,Smart TV,9
2846,Adhesives,0
2847,Mobile Toilet,6
2848,Air freshners,1


In [58]:
# function to get clean text from the product names

def clean_txt(sent):
    tokens = word_tokenize(re.sub('[^a-z0-9 ]',' ',sent.lower()))
    stop_updated = stopwords.words("english") + list(punctuation) + ["..."] + ["would", "could","told","subject"]
    text = [term for term in tokens] 
    res = " ".join(text)
    return res

In [59]:
# obtaining cleaned titles from Product_Name
df['cleaned_title'] = df['Product name'].apply(clean_txt)
df

,Product name,Category,cleaned_title
0,HP 14 11th Gen Intel Core i5 Processor 14-inch...,5,hp 14 11th gen intel core i5 processor 14 inch...
1,"Dell Vostro 3400 14"" (35.56 cms) FHD Anti Glar...",5,dell vostro 3400 14 35 56 cms fhd anti glare d...
2,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",5,hp 15 2021 thin light ryzen 3 3250 laptop 8 gb...
3,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",5,hp 15 amd athlon 15 6 39 62cms hd laptop silve...
4,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",5,asus vivobook 15 2020 39 6 cm hd dual core int...
...,...,...,...
2845,Smart TV,9,smart tv
2846,Adhesives,0,adhesives
2847,Mobile Toilet,6,mobile toilet
2848,Air freshners,1,air freshners


## Word Embedding using BERT

In [60]:
# loading Google's Pre Trainied BERT Model
model_name = "bert-base-uncased"
max_length = 512
model = RepresentationModel(
        model_type="bert",
        model_name=model_name,
        use_cuda=False
    )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [61]:
docs_vectors_bert = model.encode_sentences(df['cleaned_title'], combine_strategy='mean')

docs_vectors = pd.DataFrame(docs_vectors_bert)
docs_vectors

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.352363,-0.000895,0.635360,0.010870,0.247490,-0.004298,-0.120628,0.283842,0.106937,-0.359020,...,0.034330,-0.058128,0.135652,-0.153377,0.167897,0.152308,0.017508,-0.253459,-0.114063,-0.128271
1,-0.322499,-0.110086,0.611277,0.035415,0.336861,0.052914,-0.100935,0.232334,0.025085,-0.317202,...,0.030938,-0.077172,0.149580,-0.334034,0.257276,0.157134,-0.025339,-0.128183,-0.193576,-0.019142
2,-0.318877,0.006600,0.862213,0.015443,0.098625,-0.106705,-0.223028,0.243128,0.156237,-0.489361,...,0.129350,-0.087230,0.107937,-0.280078,0.226705,0.192856,0.013936,-0.195846,-0.054647,-0.044332
3,-0.413501,-0.012155,0.820614,0.054931,0.297246,-0.007218,-0.190128,0.127568,0.222436,-0.458878,...,-0.029261,0.008181,0.209137,-0.305906,0.247856,0.247953,-0.094879,-0.167667,-0.074621,-0.052666
4,-0.345951,-0.041854,0.726286,0.025646,0.353956,0.007568,-0.196982,0.328361,0.098349,-0.454568,...,-0.002794,-0.074385,0.132243,-0.394021,0.266071,0.312700,-0.123544,-0.228812,-0.284856,-0.149707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845,0.102012,-0.514650,0.406913,0.393420,0.142930,-0.069535,-0.096251,0.317699,0.236926,-0.381081,...,-0.064110,-0.295226,0.223565,0.200251,0.134190,0.003751,0.276813,-0.016694,0.376555,0.040392
2846,0.303796,-0.307845,-0.083208,0.006488,0.118112,0.313239,-0.286710,0.058862,0.122702,-0.406376,...,0.461161,-0.224487,0.068819,0.167735,0.317619,-0.175047,-0.487930,-0.122463,-0.312450,0.099223
2847,-0.042295,-0.118887,-0.025296,0.063842,0.236313,0.319941,-0.148707,-0.025066,-0.138942,-0.347522,...,0.624178,0.029154,0.147465,0.314092,0.007503,-0.343380,-0.316388,-0.195563,-0.399505,0.384962
2848,0.115626,-0.336022,-0.268377,0.151521,0.267100,-0.165952,0.194136,0.341228,-0.185761,-0.378783,...,0.341564,-0.414993,0.052990,-0.042663,0.168157,-0.262815,-0.100450,0.047369,-0.225666,-0.094078


## Preparing Data for modelling for data embedded using Word2Vec

In [62]:
#defining X and Y for modelling
X = docs_vectors
y = df['Category'].values

In [63]:
#Splitting X and Y into training set and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state= 2)

## Using SVM Classification Model

In [64]:
#Using SVM model for classification
svc = svm.SVC(kernel='linear')
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)


In [65]:
confusion_matrix(y_test, y_pred_svc)

array([[84,  0,  0,  1,  0,  0,  3,  0,  0,  0],
       [ 1, 82,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0, 60,  0, 23,  0,  0,  0,  0,  0],
       [ 1,  0,  0, 75,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 28,  0, 47,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  3, 80,  2,  0,  0,  0],
       [ 7,  1,  0,  2,  0,  0, 84,  1,  3,  0],
       [ 0,  0,  0,  0,  0,  0,  4, 83,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  1,  1, 76,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 99]], dtype=int64)

In [66]:
print(classification_report(y_test,y_pred_svc))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92        88
           1       0.99      0.98      0.98        84
           2       0.68      0.72      0.70        83
           3       0.96      0.99      0.97        76
           4       0.64      0.62      0.63        76
           5       1.00      0.94      0.97        85
           6       0.88      0.86      0.87        98
           7       0.98      0.95      0.97        87
           8       0.95      0.96      0.96        79
           9       1.00      1.00      1.00        99

    accuracy                           0.90       855
   macro avg       0.90      0.90      0.90       855
weighted avg       0.90      0.90      0.90       855



### Using Grid Search CV to find the best values for hyperparameters

In [67]:
# SVM Hyperparameter Tuning

#setting up parameters with their possible input.
param_grid = {'C':[0.1,1,100,1000],'kernel':['rbf','poly','sigmoid','linear'],'degree':[1,2,3,4,5,6],'gamma':['scale','auto']}

#hyperparameter_tuning on basis of SVC model and scoring metric as 'accuracy'
grid = GridSearchCV(svm.SVC(), param_grid, scoring = 'accuracy')

#fiting the training data with best parameters
grid.fit(X_train, y_train)

#predicting category for the train set after tuning the hyperparameters of the model
y_pred_grid_svm = grid.predict(X_test)

In [68]:
print(classification_report(y_test, y_pred_grid_svm))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91        88
           1       1.00      0.98      0.99        84
           2       0.70      0.71      0.71        83
           3       0.96      0.99      0.97        76
           4       0.67      0.64      0.66        76
           5       0.96      0.95      0.96        85
           6       0.89      0.87      0.88        98
           7       0.98      0.95      0.97        87
           8       0.95      0.96      0.96        79
           9       1.00      1.00      1.00        99

    accuracy                           0.90       855
   macro avg       0.90      0.90      0.90       855
weighted avg       0.90      0.90      0.90       855



In [69]:
# Listing names of the classifiers used and their scores
names = ['SVC','SVC Grid Search CV']


accuracy_scores = [accuracy_score(y_test, y_pred_svc),
                   accuracy_score(y_test, y_pred_grid_svm)]


precision_scores = [precision_score(y_test, y_pred_svc, average = 'macro'),
                    precision_score(y_test, y_pred_grid_svm, average = 'macro')]

recall_scores = [recall_score(y_test, y_pred_svc, average = 'macro'),
                 recall_score(y_test, y_pred_grid_svm, average = 'macro')]

f1_scores = [f1_score(y_test, y_pred_svc, average = 'macro'),
             f1_score(y_test, y_pred_grid_svm, average = 'macro')]

In [70]:
# Getting a dataset from the lists above to easily compare the scores of the model used
scores_df = pd.DataFrame(zip(names, accuracy_scores, precision_scores, recall_scores, f1_scores), 
                           columns = ['Classifier', 'Accuracy Score', 'Precision Score', 'Recall Score', 'F1 Score'])
scores_df

,Classifier,Accuracy Score,Precision Score,Recall Score,F1 Score
0,SVC,0.900585,0.897944,0.897326,0.897302
1,SVC Grid Search CV,0.902924,0.899430,0.899813,0.899433


#### After observing the performance of the models in fixed random_state and random text splittling, we concluded that only Gaussian Naive Bayes and SVM classifiers are stable in their preformance regardless of the randomness of the test data, hence we use both the classifiers henceforth.

## Getting predicted Category of entered Product Name

In [71]:
# UDF to clean the input text and get its vectorised form

def vect(sent):
    # sends text to clean using clean_txt()
    cln = clean_txt(sent)
    docs_vectors_bert = model.encode_sentences([cln], combine_strategy='mean')
    docs_vectors = pd.DataFrame(docs_vectors_bert)
    return docs_vectors

In [72]:
# UDF to get the class with most number of occurrences in the pred array
def most_likely(sent):
    class_ = []
    counter = 0
    new = list(label_encoder.inverse_transform(sent))
    for i in new:
        curr_frequency = new.count(i)
        if(curr_frequency > counter):
            counter = curr_frequency
            class_ = i
    return class_ 

In [81]:
#Get input from the user and sending that to UDF above
inp = vect(input('Enter title:'))

#predicting the category from user input
pred_svm = svc.predict(inp)
pred_svm_grid = grid.predict(inp)
#printing the category predicted using UDF most_likely()
print('\nCategory SVM:', most_likely(pred_svm))
print('\nCategory SVM:', label_encoder.inverse_transform(pred_svm_grid)[0])

Enter title:acer

Category SVM: Air freshners

Category SVM: Air freshners


In [74]:
import joblib
joblib.dump(svc,"svc.category_classifier.pkl")

['svc.category_classifier.pkl']